In [ ]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.spatial import cKDTree
from scipy.stats import binned_statistic
from scipy.interpolate import interp1d
from tqdm import tqdm

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from totoro.config import galcen_frame, elem_names
from totoro.data import datasets
from totoro.potentials import potentials, galpy_potentials
from totoro.objective import TorusImagingObjective

In [ ]:
# t, c = load_apogee_sample('../data/apogee-parent-sample.fits')
# lb0_mask = c.separation(coord.SkyCoord(l=0, b=0, unit=u.deg, frame='galactic')) > 45*u.deg
# t = t[lb0_mask]
# c = c[lb0_mask]

# c = c[t['GAIA_PARALLAX'] > 0.666]
# t = t[t['GAIA_PARALLAX'] > 0.666]
# len(t)

In [ ]:
data_name = 'apogee-rgb-loalpha'
d = datasets[data_name]
galcen = d.c.transform_to(galcen_frame)
w0s = gd.PhaseSpacePosition(galcen.data)

In [ ]:
stuff = {}
for check_elem in ['MG_FE', 'SI_FE', 'FE_H']:
    stuff[check_elem] = {}
    for i, namey in zip([0, 1, 2],
                        ['mdisk', 'zsun', 'vzsun']):
        all_p_vals = []
        all_obj_vals = []
        tree_Ks = [8, 16, 20, 32, 64, 128]
        for tree_K in tree_Ks:
            obj = TorusImagingObjective(d, check_elem, tree_K=tree_K)

            x0 = np.array([1.1, 20.8, 7.78])
            # x0 = res.x

            idx = np.arange(3)
            idx = np.delete(idx, i)

            obj_vals = [] 
            
            NN = 21
            if i == 0:
                p_vals = np.linspace(0.4, 1.6, NN)
            elif i == 1:
                p_vals = np.linspace(-40, 40, NN)
            elif i == 2:
                p_vals = np.linspace(-5, 20, NN)
                
            for val in tqdm(p_vals):
                p = np.full(3, np.nan)
                p[i] = val
                p[idx] = x0[idx]
                obj_vals.append(obj(p))

            obj_vals = np.array(obj_vals)
            all_p_vals.append(p_vals)
            all_obj_vals.append(obj_vals)
            
        stuff[check_elem][namey] = (np.array(all_p_vals), 
                                    np.array(all_obj_vals))

In [ ]:
check_elem = 'MG_FE'
s = stuff[check_elem]

In [ ]:
for check_elem in stuff.keys():
    # check_elem = 'MG_FE'
    s = stuff[check_elem]

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    for i, name in enumerate(s.keys()):
        ax = axes[i]
        for tree_K, p_vals, obj_vals in zip(tree_Ks, *s[name]):
            l, = ax.plot(p_vals, obj_vals * 1e7, label=f"K={tree_K}")

            obj_vals = np.array(obj_vals)
            ax.plot(p_vals[np.nanargmin(obj_vals)], np.nanmin(obj_vals) * 1e7,
                    color=l.get_color(), marker='o', ms=8)

    ax.legend()
    fig.set_facecolor('w')
    fig.tight_layout()

stuff

In [ ]:
tree_Ks = 2 ** np.arange(2, 8+1, 1)
results = []
for tree_K in tqdm(tree_Ks):
    obj = TorusImagingObjective(t, c, 'MG_FE', tree_K=tree_K)
    res = obj.minimize()
    results.append(res)

In [ ]:
xs = np.array([r.x for r in results])

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(8, 10), 
                         sharex=True)
for i in range(3):
    axes[i].plot(tree_Ks, xs[:, i])
    axes[i].errorbar(tree_Ks, xs[:, i], 0.07*xs[:, i], 
                     zorder=-10, marker='', ls='none', ecolor='#aaaaaa')

axes[0].set_ylabel('mdisk')
axes[1].set_ylabel('zsun')
axes[2].set_ylabel('vzsun')
    
axes[0].set_title('optimization results')
axes[0].set_xscale('log', basex=2)
axes[2].set_xlabel('$K$')
fig.set_facecolor('w')

### Bootstrap testing:

In [ ]:
tree_K = 20
boot_K = 128
np.random.seed(42)

# Full sample fit:
# obj = TorusImagingObjective(t, c, 'MG_FE', tree_K=tree_K)
# full_sample_res = obj.minimize(method="nelder-mead", options=dict(maxiter=4))

obj = TorusImagingObjective(t, c, 'SI_FE', tree_K=tree_K)
full_sample_res = obj.minimize(method="nelder-mead")

if not full_sample_res.success:
    raise RuntimeError("WTF")

In [ ]:
full_sample_res

In [ ]:
import scipy
scipy.__version__

In [ ]:
results = []
for k in tqdm(range(boot_K)):
    idx = np.random.choice(len(t), len(t), replace=True)
    obj = TorusImagingObjective(t[idx], c[idx], 'MG_FE', 
                                tree_K=tree_K)
    res = obj.minimize(x0=full_sample_res.x)
    results.append(res)

In [ ]:
results[0]

In [ ]:
full_sample_res

---

In [ ]:
minimize(func, x0=1.5, method='nelder-mead')